In [8]:
import requests,time,os
import tkinter as tk
from tkinter import ttk
from selenium import webdriver
from bs4 import BeautifulSoup
from PIL import Image,ImageTk

#===顯示漫畫內容函式
def show_content():
    ep = eps_dp.get()
    if ep != '請選擇':
        ep_num = ep.strip('第').strip('話')
        newurl = url2 + ep_num
        options = webdriver.ChromeOptions()
        options.add_argument("headless") #無視窗模式
        driver = webdriver.Chrome('chromedriver',options=options)
        driver.get(newurl)

        soup = BeautifulSoup(driver.page_source,'lxml')
        dropdown = soup.select_one('#pageindex')
        pages = dropdown.select('option')
        pp = len(pages)
        
        path = f'c://comic/{ep_num}/'
        if not os.path.isdir(path):
            findjpg(ep_num, pp, newurl)
        
        top = tk.Toplevel()
        top.title(bookname.text + ep)
        top.geometry('450x500')
        top.iconbitmap('logo.ico')
        main_frame = tk.Frame(top)
        main_frame.pack(fill=tk.BOTH, expand=1)
        
        my_canvas = tk.Canvas(main_frame)
        my_canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)
        
        my_scrollbar = ttk.Scrollbar(main_frame, orient=tk.VERTICAL, command=my_canvas.yview)
        my_scrollbar.pack(side=tk.LEFT, fill=tk.Y)
        
        my_canvas.configure(yscrollcommand=my_scrollbar.set)
        my_canvas.bind('<Configure>', lambda e: my_canvas.configure(scrollregion = my_canvas.bbox("all")))
        
        second_frame = tk.Frame(my_canvas)
        
        my_canvas.create_window((0,0), window = second_frame, anchor="nw")
        
        LB1 = tk.Label(second_frame, text=ep, font='Arial 15 bold')
        LB1.pack()

        LB2 = tk.Label(second_frame, text=f'總共{pp}頁', font='Arial 15 bold')
        LB2.pack()

        #===顯示圖片 
        for j in range(1,pp+1):
            locals()['img'+str(j)] = Image.open(path + str(j) + '.jpg')

            width, height = locals()['img'+str(j)].size
            locals()['img'+str(j)] = locals()['img'+str(j)].resize((width//2, height//2)) 

            locals()['tk_img'+str(j)] = ImageTk.PhotoImage(locals()['img'+str(j)]) #把一般照片img轉成tk模式的照片

            locals()['ImageLabel'+str(j)] = tk.Label(second_frame, image=locals()['tk_img'+str(j)])
            locals()['ImageLabel'+str(j)].pack()
            
            
        top.mainloop()
    else:
        top = tk.Toplevel()
        top.title("請選擇")
        top.geometry('400x100')
        top.iconbitmap('logo.ico')
        
        LB1 = tk.Label(top, text='沒有選擇集數，請重新選擇第幾話!', font='Arial 15 bold',fg='red')
        LB1.pack()
        
        top.mainloop()
    
#===下載圖片函式   
def findjpg(ep, p, firsturl):
    for i in range(1, p+1):
        finurl = firsturl + '-' + str(i)
        options = webdriver.ChromeOptions()
        options.add_argument("headless") #無視窗模式
        driver = webdriver.Chrome('chromedriver',options=options)
        driver.get(finurl)
        time.sleep(1)

        soup = BeautifulSoup(driver.page_source,'lxml')
        imgurl = soup.select_one('#TheImg').get('src')
        fimgurl = 'https:' + imgurl

        r = requests.get(fimgurl)
        path = f'c://comic/{ep}/'
        if not os.path.isdir(path):
            os.mkdir(path)
        with open(f'c://comic/{ep}/{i}.jpg', 'wb') as f:
            f.write(r.content) #r.content專找圖片
    
url = 'https://www.comicabc.com/html/16533.html'
url2 = 'https://a.twobili.com/online/new-16533.html?ch='
r = requests.get(url)
r.encoding = 'utf-8'
soup = BeautifulSoup(r.text,'lxml')

#===抓取漫畫書資訊
bookname = soup.find('h3', class_='item_name')
eps = soup.find('a', class_='item_comic_eps')
update = soup.find_all('span', class_='font_small')[1]



#===主視窗
root = tk.Tk()
root.title('自製電子書')
root.geometry('600x100')
root.iconbitmap('logo.ico')
bookname_LB = tk.Label(root, text='書名 : ' + bookname.text.strip('\n'), font='Arial 15', width=15)
bookname_LB.grid(column=0, row=0)

eps_LB = tk.Label(root, text='集數 : ' + eps.text, font='Arial 15', width=15)
eps_LB.grid(column=1, row=0)

update_LB = tk.Label(root, text='更新日期 : ' + update.text, font='Arial 15', width=20)
update_LB.grid(column=2, row=0)

eps_split = eps.text.split('-')
first = int(eps_split[0])
final = int(eps_split[1])

nums = ['請選擇']
for i in range(first, final+1):
    nums.append(f'第{i}話')
sel_nums = tk.StringVar()
eps_dp = ttk.Combobox(root, textvariable=sel_nums, values=nums, font='Arial 15', width='15')
eps_dp.current(0)
eps_dp.grid(column=1, row=1)

btn1 = tk.Button(root, text='閱讀', font='Arial 15', command=show_content)
btn1.grid(column=2, row=1)

root.mainloop()